In [27]:
lines = []

with open("../data/day_3__data.txt", "r") as file:
    for line in file.readlines():
        lines.append(line.replace('\n', ''))

print(lines)

['...788.............................54.........501...........555.........270.................................521......893....................', '..../..*963........................*..860......................*....53...../.....................52.................&....347........428*522.', '............*......41..481+.......462....$..187......678.......420....-....................&115.+...........................+...............', '............707....&.........562...........*..........*.....................438....................&877..660....199..145.........71.........', '.....210................356..*.........977.68.........38.......835.622.332.....*300.....131.422..............89..*.....+..........$.........', '..............14..312......+..926.....*.......529..*............*...*....*.............*......%...310.......*...835..................885....', '...416../467..........................423.....*...143...132..955...356...124.........588..947....*.....512......................134&.*

In [20]:
from dataclasses import dataclass

@dataclass
class Coord:
    x: int
    y: int

    def __init(self, x: int, y: int):
        self.x = x
        self.y = y

In [21]:
from enum import Enum

class CellType(Enum):
    NUMBER = 1,
    DOT = 2,
    SYMBOL = 3,
    GEAR = 4

    @classmethod
    def from_cell(cls, cell):
        if cell.isnumeric():
            return CellType.NUMBER
        elif cell == '.':
            return CellType.DOT
        elif cell == '*':
            return CellType.GEAR
        else:
            return CellType.SYMBOL

In [29]:
def is_valid_coord(coord: Coord, min: Coord, max: Coord) -> bool:
    return coord.x >= min.x and coord.x < max.x and coord.y >= min.y and coord.y < max.y

def get_adjacent_coords(coord: Coord, min: Coord, max: Coord) -> list: 
    adjacents = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            adjacent = Coord(coord.x + i, coord.y + j)
            if is_valid_coord(adjacent, min, max):
                adjacents.append(adjacent)
    return adjacents

def get_valid_numbers(lines: list) -> list:
    valids = []
    current_number = None
    valid = False

    min = Coord(0, 0)
    max = Coord(len(lines), len(lines[0]))

    for line_index, line in enumerate(lines):
        valids_on_line = []
        for cell_index, cell in enumerate(line):
            cell_type = CellType.from_cell(cell)
            if cell_type == CellType.NUMBER:
                if current_number == None:
                    current_number = cell
                else:
                    current_number += cell
                if not valid:
                    adjacents = get_adjacent_coords(Coord(line_index, cell_index), min, max)
                    for a in adjacents:
                        adj_type = CellType.from_cell(lines[a.x][a.y])
                        if adj_type == CellType.SYMBOL or adj_type == CellType.GEAR:
                            valid = True
                            break
            else:
                if current_number is not None:
                    if valid:
                        valids_on_line.append(int(current_number))
                    current_number = None
                    valid = False
        if current_number is not None:
            if valid:
                valids_on_line.append(int(current_number))
            current_number = None
            valid = False

        print(line, valids_on_line)
        valids.extend(valids_on_line)
      
    return valids

def get_number_at_coord(lines: list, coord: Coord) -> int:
    num_cell = Coord(coord.x, coord.y)

    while ((CellType.from_cell(lines[num_cell.x][num_cell.y - 1]) == CellType.NUMBER)) and (num_cell.y - 1 >= 0):
        num_cell = Coord(num_cell.x, num_cell.y - 1)
    
    str_num = ""
    while (True):
        str_num += lines[num_cell.x][num_cell.y]
        num_cell = Coord(num_cell.x, num_cell.y + 1)
        if num_cell.y >= len(lines[coord.y]) or CellType.from_cell(lines[num_cell.x][num_cell.y]) != CellType.NUMBER:
            break
        

    return int(str_num)

def get_gears_ratio(lines: list) -> list:
    ratios = []

    min = Coord(0, 0)
    max = Coord(len(lines), len(lines[0]))

    for line_index, line in enumerate(lines):
        ratios_on_line = []
        for cell_index, cell in enumerate(line):
            cell_type = CellType.from_cell(cell)
            if cell_type == CellType.GEAR:
                # Oooooh je triiiiche c'est maaaaal
                # Si j'ai deux nombres identiques adjacents d'un GEAR alors ils seront considérés comme un seul
                # Mais bon ça passe !
                adjacents_numbers = set()
                adjacents = get_adjacent_coords(Coord(line_index, cell_index), min, max)                
                for adj in adjacents:
                    for a in adjacents:
                        adj_type = CellType.from_cell(lines[a.x][a.y])
                        if adj_type == CellType.NUMBER:
                            adjacents_numbers.add(get_number_at_coord(lines, a))
                if len(adjacents_numbers) == 2:
                    ratios_on_line.append(list(adjacents_numbers)[0] * list(adjacents_numbers)[1])

        print(line, ratios_on_line)
        ratios.extend(ratios_on_line)
    
    return ratios


#print(sum(get_valid_numbers(lines)))
print(sum(get_gears_ratio(lines)))

...788.............................54.........501...........555.........270.................................521......893.................... []
..../..*963........................*..860......................*....53...../.....................52.................&....347........428*522. [24948, 233100, 223416]
............*......41..481+.......462....$..187......678.......420....-....................&115.+...........................+............... []
............707....&.........562...........*..........*.....................438....................&877..660....199..145.........71......... [12716, 25764]
.....210................356..*.........977.68.........38.......835.622.332.....*300.....131.422..............89..*.....+..........$......... [520412, 131400, 166165]
..............14..312......+..926.....*.......529..*............*...*....*.............*......%...310.......*...835..................885.... [413271, 797425, 221432, 41168, 77028]
...416../467..........................423....